In [23]:
# from timm import create_model

import torchvision
import torchvision.transforms as transforms

from models import *
from utils import fit, torch_seed, progress_bar
import os
import torch
import torch.nn as nn

from copy import deepcopy

import numpy as np

from glob import glob

import pandas as pd

# Config


In [24]:
class Config:
    datadir = './data/'
    # modelname = 'vit_tiny_patch16_224_in21k'
    batch_size=128
    img_size = 224
    
args = Config()

## models path


In [25]:
#models_path =  './logs/provisional_results/simplecnn_*/*/*/student.pt'
models_path =  './logs/simplecnn_*/*/*/student.pt'

# Setting the seed

In [26]:
torch_seed(223)
device = torch.device(f'cuda' if torch.cuda.is_available() else 'cpu')

# DataLoader


In [27]:
def build_loader(datadir, batch_size):
    transform_test = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])


    testset = torchvision.datasets.CIFAR10(
        root=datadir, train=False, download=True, transform=transform_test)
    testloader = torch.utils.data.DataLoader(
        testset, batch_size=batch_size, shuffle=False, num_workers=6)

    return testloader

# def test(net, testloader):
#     criterion = nn.CrossEntropyLoss()
    
#     global best_acc
#     net.eval()
#     test_loss = 0
#     correct = 0
#     total = 0
#     with torch.no_grad():
#         for batch_idx, (inputs, targets) in enumerate(testloader):
#             inputs, targets = inputs.cuda(), targets.cuda()
#             outputs = net(inputs)
#             loss = criterion(outputs, targets)

#             test_loss += loss.item()
#             _, predicted = outputs.max(1)
#             total += targets.size(0)
#             correct += predicted.eq(targets).sum().item()
    
#     acc = 100.*correct/total
#     loss = test_loss / len(testloader)
    
#     return acc, loss

def test(model, dataloader, criterion=nn.CrossEntropyLoss()):
    global best_acc
    model.eval()
    correct = 0
    total = 0
    total_loss = 0


    with torch.no_grad():
        for idx, (inputs, targets) in enumerate(dataloader):
            inputs, targets = inputs.cuda(), targets.cuda()
            
            # predict
            outputs = model(inputs)
            
            # loss 
            loss = criterion(outputs, targets)
            
            # total loss and acc
            total_loss += loss.item()
            preds = outputs.argmax(dim=1)
            correct += targets.eq(preds).sum().item()
            total += targets.size(0)
            
            # #massage
            # progress_bar(current=idx, 
            #              total=len(dataloader),
            #              msg='Loss: %.3f | Acc: %.3f%% [%d/%d]' % (total_loss/(idx + 1), 
            #                                                      100.*(correct/total), correct, total),
            #              term_width=100)

    return correct/total, total_loss/len(dataloader)

# Model Soup

In [28]:
def add_ingradient(soup, ckp_path):
    #ingradient = create_model(modelname)
    ingradient = Student()
    ingradient.load_state_dict(torch.load(ckp_path)['model'])
    ingradient.cuda()

    for param1, param2 in zip(soup.parameters(), ingradient.parameters()):
        param1.data = (param1.data + param2.data) / 2
        
    return soup

def greedy_soup(ckp_list, testloader):
    #soup = create_model(modelname)
    soup = Student()
    soup.load_state_dict(torch.load(ckp_list[0])['model'])
    soup.cuda()
    
    best_acc, loss = test(soup, testloader)
    print(f'first acc: {best_acc:.2f}%')
    
    # cook
    for i, ckp_i in enumerate(ckp_list[1:]):
        soup_next = deepcopy(soup)
        soup_next = add_ingradient(soup_next, ckp_i)
        acc, loss = test(soup_next, testloader)
        
        print(f'acc of {i} ingradient: {acc:.2f}%')
        
        if acc > best_acc:
            soup = soup_next
    
    return soup

# Results



### Simple CNN


In [29]:
testloader = build_loader(args.datadir, args.batch_size)

Files already downloaded and verified


In [30]:
ckp_list = glob(models_path)
print(len(ckp_list))
acc_list = []
loss_list = []

for ckp_i in ckp_list:
    #Student is defined in models...
    net = Student()
    # net = create_model(args.modelname)
    net.cuda()
    # load the student state at the given checkpoint
    net.load_state_dict(torch.load(ckp_i)['model'])

    acc, loss = test(net, testloader)
    
    acc_list.append(acc)
    loss_list.append(loss)
    
# cook
ckp_list =  glob(models_path)
net_soup = greedy_soup(ckp_list, testloader)

acc, loss = test(net_soup, testloader)
    
acc_list.append(acc)
loss_list.append(loss)
ckp_list.append('soup')

9
first acc: 0.61%
acc of 0 ingradient: 0.60%
acc of 1 ingradient: 0.62%
acc of 2 ingradient: 0.62%
acc of 3 ingradient: 0.59%
acc of 4 ingradient: 0.62%
acc of 5 ingradient: 0.62%
acc of 6 ingradient: 0.61%
acc of 7 ingradient: 0.57%


In [31]:
simpleCNN_df = pd.DataFrame({'acc':acc_list, 'loss':loss_list, 'ckp_path':ckp_list})
simpleCNN_df 

,acc,loss,ckp_path
0,0.6124,1.125507,./logs/simplecnn_0.06/method1/KD_student_alpha...
1,0.6152,1.117879,./logs/simplecnn_0.03/method1/KD_student_alpha...
2,0.6074,1.154185,./logs/simplecnn_0.09/method1/KD_student_alpha...
3,0.6083,1.134483,./logs/simplecnn_0.07/method1/KD_student_alpha...
4,0.6102,1.122326,./logs/simplecnn_0.02/method1/KD_student_alpha...
5,0.6151,1.120879,./logs/simplecnn_0.05/method1/KD_student_alpha...
6,0.6094,1.140904,./logs/simplecnn_0.08/method1/KD_student_alpha...
7,0.6126,1.118564,./logs/simplecnn_0.04/method1/KD_student_alpha...
8,0.5988,1.152616,./logs/simplecnn_0.01/method1/KD_student_alpha...
9,0.6216,1.112639,soup


In [32]:
simpleCNN_df.round(4).to_csv('simple-cnn-results.csv',index=False)